In [35]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import io
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pdb

BATCH_SIZE = 32

### Using FastText Word Embeddings

In [36]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    print(n, d)
    i = 0
    for line in fin:
        if i == VOCAB_SIZE:
            break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
        i += 1
    return data

### Build vocabulary, id2token, token2id

In [37]:
from collections import Counter
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = 50000

def build_vocab():
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    word_vectors = load_vectors("/Users/vrajiv/Desktop/rnn-cnn-natural-language-inference/wiki-news-300d-1M-subword.vec")
    id2token = list(word_vectors.keys())
    token2id = dict(zip(word_vectors, range(2,2+len(word_vectors)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return word_vectors, token2id, id2token

In [38]:
word_vectors, token2id, id2token = build_vocab()

999994 300


In [39]:
_weights = np.array(list(word_vectors.values()))
pad_vec = np.zeros((1, 300))
unk_vec = np.random.randn(1, 300) * 0.01
pad_unk_vecs = np.vstack((pad_vec, unk_vec))
_WEIGHTS = np.vstack((pad_unk_vecs, _weights))
_WEIGHTS.shape

(50002, 300)

### Creating toy dataset with 5000 training data, 1000 validation

In [102]:
snli_train = pd.read_csv('/Users/vrajiv/Desktop/rnn-cnn-natural-language-inference/hw2_data/snli_train.tsv', sep='\t')
TRAIN_VAL_SPLIT = 20000
DATA_SIZE = 25000

In [103]:
sent1_data = list(snli_train["sentence1"])[:DATA_SIZE]
sent2_data = list(snli_train["sentence2"])[:DATA_SIZE]
data_label = list(snli_train["label"])[:DATA_SIZE]
print(len(sent1_data))

25000


In [104]:
def convert_labels_to_integers(data_label):
    for i in range(len(data_label)):
        if data_label[i] == "contradiction":
            data_label[i] = 0
        elif data_label[i] == "entailment":
            data_label[i] = 1
        elif data_label[i] == "neutral":
            data_label[i] = 2
    return data_label

In [105]:
data_label = convert_labels_to_integers(data_label)

In [106]:
SEED = 1
random.Random(SEED).shuffle(sent1_data)
random.Random(SEED).shuffle(sent2_data)
random.Random(SEED).shuffle(data_label)

In [107]:
def verify_order(sent1_data, sent2_data, data_label):
    i = random.randint(1, len(sent1_data))
    print(sent1_data[i])
    print(sent2_data[i])
    print(data_label[i])

verify_order(sent1_data, sent2_data, data_label)

A mountain climber wearing an orange helmet swings his Axe .
A mountain climber in a helmet swings his axe as he approaches the top of the mountain .
2


### Train/Val split

In [108]:
sent1_train = sent1_data[:TRAIN_VAL_SPLIT]
sent2_train = sent2_data[:TRAIN_VAL_SPLIT]
train_label = data_label[:TRAIN_VAL_SPLIT]

sent1_val = sent1_data[TRAIN_VAL_SPLIT:]
sent2_val = sent2_data[TRAIN_VAL_SPLIT:]
val_label = data_label[TRAIN_VAL_SPLIT:]

In [109]:
print(len(sent1_train), len(sent2_train), len(train_label))
print(len(sent1_val), len(sent2_val), len(val_label))

20000 20000 20000
5000 5000 5000


In [110]:
verify_order(sent1_train, sent2_train, train_label)
verify_order(sent1_val, sent2_val, val_label)

one woman holding flowers and two men on a bench
A woman is holding flowers by two men on a bench .
1
Young african american child is held and stars at the camera .
Black child held and staring at the camera
1


### Word tokenizing

In [111]:
def tokenize(sentence_list):
    return [word_tokenize(sentence_list[i]) for i in range(len(sentence_list))]

# train
sent1_train_tokenized = tokenize(sent1_train)
sent2_train_tokenized = tokenize(sent2_train)

# val
sent1_val_tokenized = tokenize(sent1_val)
sent2_val_tokenized = tokenize(sent2_val)

### "One-hot encoding"
#### Represent each sentence as a vector of indices in our vocabulary

In [112]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data 

In [113]:
# train
sent1_train_indices = token2index_dataset(sent1_train_tokenized)
sent2_train_indices = token2index_dataset(sent2_train_tokenized)

# val
sent1_val_indices = token2index_dataset(sent1_val_tokenized)
sent2_val_indices = token2index_dataset(sent2_val_tokenized)

### Data Loader

In [114]:
MAX_SENTENCE_LENGTH = 300

import numpy as np
import torch
from torch.utils.data import Dataset

class TwoSentencesDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    """
    
    def __init__(self, sent1_data_list, sent2_data_list, target_list):
        """
        @param sent1_data_list: list of sentence1's (index matches sentence2's and target_list below)
        @param sent2_data_list: list of sentence2's
        @param target_list: list of correct labels

        """
        self.sent1_data_list = sent1_data_list
        self.sent2_data_list = sent2_data_list
        self.target_list = target_list
        assert (len(self.sent1_data_list) == len(self.target_list) and len(self.sent2_data_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_data_list)
        
    def __getitem__(self, key):
        ###
        ### Returns [[sentence, 1, tokens], [sentence, 2, tokens]]
        ###
        """
        Triggered when you call dataset[i]
        """
        sent1_tokens_idx = self.sent1_data_list[key][:MAX_SENTENCE_LENGTH]
        sent2_tokens_idx = self.sent2_data_list[key][:MAX_SENTENCE_LENGTH]
        combined_tokens_idx = [sent1_tokens_idx, sent2_tokens_idx]
        label = self.target_list[key]
        return [combined_tokens_idx, len(sent1_tokens_idx), len(sent2_tokens_idx), label]

def twosentences_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    sent1_data_list = []
    sent2_data_list = []
    sent1_length_list = []
    sent2_length_list = []
    label_list = []
    combined_data_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[3])
        sent1_length_list.append(datum[1])
        sent2_length_list.append(datum[2])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        combined_data_list.append([padded_vec_1, padded_vec_2])
    return [torch.from_numpy(np.array(combined_data_list)), 
            torch.LongTensor(sent1_length_list), torch.LongTensor(sent2_length_list), torch.LongTensor(label_list)]

In [115]:
train_dataset = TwoSentencesDataset(sent1_train_indices, sent2_train_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

### The Model

In [116]:
class TwoSentenceModel(nn.Module):
    def __init__(self, hidden_size, num_layers, num_classes, emb_size = 300):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # num_classes: number of output classes
        # vocab_size: vocabulary size
        super(TwoSentenceModel, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        weight = torch.FloatTensor(_WEIGHTS)
        self.embedding = nn.Embedding.from_pretrained(weight, freeze=True)
#         self.rnn = nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
#         self.linear = nn.Linear(2*hidden_size, num_classes)
        # TRYING GRU, UNCOMMENT Below if doing GRU
        self.rnn = nn.GRU(emb_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(4*hidden_size, num_classes)

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        return torch.randn(2, batch_size*2, self.hidden_size)

    def forward(self, x, sent1_lengths, sent2_lengths):
        # reset hidden state
        batch_size = x.size()[0]
                
        s1lengths = list(sent1_lengths)
        s2lengths = list(sent2_lengths)
        ordered_slengths = s1lengths + s2lengths

        reverse_sorted_indices = [x for _, x in sorted(zip(ordered_slengths, range(len(ordered_slengths))), reverse=True)]
        reverse_sorted_lengths = [x for x, _ in sorted(zip(ordered_slengths, range(len(ordered_slengths))), reverse=True)]
        reverse_sorted_lengths = np.array(reverse_sorted_lengths)
        
        sent1s = x[:, 0, :]
        sent2s = x[:, 1, :]
        ordered_sents = torch.cat([sent1s, sent2s], dim=0)
        reverse_sorted_data = torch.index_select(ordered_sents, 0, torch.tensor(reverse_sorted_indices))
        
        # get embedding
        embed = self.embedding(reverse_sorted_data)
        
        self.hidden = self.init_hidden(batch_size)

        # pack padded sequence
        embed = torch.nn.utils.rnn.pack_padded_sequence(embed, reverse_sorted_lengths, batch_first=True)
              
        # fprop though RNN
        rnn_out, self.hidden = self.rnn(embed, self.hidden)
                
        ### MATCHING BACK
        change_it_back = [x for _, x in sorted(zip(reverse_sorted_indices, range(len(reverse_sorted_indices))))]
        self.hidden = torch.index_select(self.hidden, 1, torch.LongTensor(change_it_back)) 
        
        # 2 by 64 by 250. back in the right order that it came in. 
        
#         hidden_sent1s = self.hidden[0, 0:batch_size, :]
#         hidden_sent2s = self.hidden[0, batch_size:, :]
              
        ### GRU stuff
        hidden_sent1s = torch.cat([self.hidden[0, 0:batch_size, :], self.hidden[1, 0:batch_size, :]], dim=1)
        hidden_sent2s = torch.cat([self.hidden[0, batch_size:, :], self.hidden[1, batch_size:, :]], dim=1)
                
        logits = self.linear(torch.cat([hidden_sent1s, hidden_sent2s], dim=1))
        return logits

In [117]:
b = torch.randn(32, 250)
a = torch.randn(32, 250)
torch.cat([a, b], dim = 1).shape

torch.Size([32, 500])

In [118]:
# Function for testing the model
def test_model(loader, model):
    """
    Helper function that tests the model's performance on a dataset
    """
    correct = 0
    total = 0
    model.eval()
    for (data, sent1_lengths, sent2_lengths, labels) in loader:
        data_batch, sent1_length_batch, sent2_length_batch, label_batch = data, sent1_lengths, sent2_lengths, labels
        outputs = F.softmax(model(data_batch, sent1_length_batch, sent2_length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def train_model(model, lr = 0.001, num_epochs = 5, criterion = nn.CrossEntropyLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

    for epoch in range(num_epochs):
        for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, sent1_length_batch, sent2_length_batch, label_batch = data, sent1_lengths, sent2_lengths, labels
            optimizer.zero_grad()
            outputs = model(data_batch, sent1_length_batch, sent2_length_batch)
            loss = criterion(outputs, label_batch)
            
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc))
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), loss))

In [119]:
model = TwoSentenceModel(emb_size = 300, hidden_size=250, num_layers=1, num_classes=3)
train_model(model)

Epoch: [1/5], Step: [101/625], Validation Acc: 40.44
Epoch: [1/5], Step: [101/625], Training Loss: 1.052436113357544
Epoch: [1/5], Step: [201/625], Validation Acc: 44.36
Epoch: [1/5], Step: [201/625], Training Loss: 1.1289901733398438
Epoch: [1/5], Step: [301/625], Validation Acc: 45.32
Epoch: [1/5], Step: [301/625], Training Loss: 0.9672324061393738
Epoch: [1/5], Step: [401/625], Validation Acc: 48.2
Epoch: [1/5], Step: [401/625], Training Loss: 0.9600820541381836
Epoch: [1/5], Step: [501/625], Validation Acc: 50.44
Epoch: [1/5], Step: [501/625], Training Loss: 1.05705988407135
Epoch: [1/5], Step: [601/625], Validation Acc: 49.42
Epoch: [1/5], Step: [601/625], Training Loss: 0.9790533781051636
Epoch: [2/5], Step: [101/625], Validation Acc: 52.52
Epoch: [2/5], Step: [101/625], Training Loss: 1.04009211063385
Epoch: [2/5], Step: [201/625], Validation Acc: 52.76
Epoch: [2/5], Step: [201/625], Training Loss: 0.9470536708831787
Epoch: [2/5], Step: [301/625], Validation Acc: 52.82
Epoch: [2

In [120]:
sample_sent1 = ['How', "are", "you", "doing", "my", "main", "man"]
sample_sent2 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]
sample_token_data = [sample_sent1, sample_sent2]

indices_data = torch.from_numpy(np.array(token2index_dataset(sample_token_data)))
model.forward(indices_data, lengths = torch.from_numpy(np.array([7, 7])))

TypeError: forward() got an unexpected keyword argument 'lengths'

In [ ]:
sample_sent1 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent2 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_sent3 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent4 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_sent5 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent6 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_combined_list = []
sample_combined_list.append([sample_sent1, sample_sent2])
sample_combined_list.append([sample_sent3, sample_sent4])
sample_combined_list.append([sample_sent5, sample_sent6])


np.array(sample_combined_list).shape

In [ ]:
for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
    print("DATA\n")
    print(data.size())
    print (data)
    print(sent1_lengths)
    print(sent2_lengths)
    print (labels)
    break

In [ ]:
###
### Initialize unk vector to * 0.01 and 0.001.  
###

###
### Sort descending before doing pack_padded_sequence
###

###
### Sort all training sentences by descending order. 
### Feed it into the batch. 
### Run your model through each of them to get hidden outputs
### Match them back again to right order to calculate loss. 
###

###
### Freeze the embedding matrix weights so they don't train. 
###